# Coding Dojo: Azure OpenAI service for GPTs

## Install Dependencies

In [1]:
!pip install openai
!pip install python-dotenv
# Only required for the explanation of the tokenization
!pip install tiktoken


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Environment Setup
Before executing the following cells, make sure to set the `AZURE_OPENAI_KEY` and `AZURE_OPENAI_ENDPOINT` variables in the `.env` file.

<br/>
<img src="assets/keys_endpoint.png" width="800"/>


In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-03-15-preview' # may change in the future

Using the `Deployment` API we can see what are the models deployed in our subscription as well as access additional information such as the actual model's name and the owner

In [4]:
deployments = openai.Deployment.list().data
print(deployments)

[<Deployment deployment id=gpt-35-dojo at 0x23c32cf5db0> JSON: {
  "created_at": 1684879507,
  "id": "gpt-35-dojo",
  "model": "gpt-35-turbo",
  "object": "deployment",
  "owner": "organization-owner",
  "scale_settings": {
    "scale_type": "standard"
  },
  "status": "succeeded",
  "updated_at": 1684917329
}, <Deployment deployment id=davinci-3 at 0x23c32cf54f0> JSON: {
  "created_at": 1684890004,
  "id": "davinci-3",
  "model": "text-davinci-003",
  "object": "deployment",
  "owner": "organization-owner",
  "scale_settings": {
    "scale_type": "standard"
  },
  "status": "succeeded",
  "updated_at": 1684890004
}, <Deployment deployment id=test-davinci-2 at 0x23c5a880900> JSON: {
  "created_at": 1685377126,
  "id": "test-davinci-2",
  "model": "text-davinci-002",
  "object": "deployment",
  "owner": "organization-owner",
  "scale_settings": {
    "scale_type": "standard"
  },
  "status": "succeeded",
  "updated_at": 1685377126
}]


If the name of the deployed model is known in advance, the lookup can be done more efficiently with the `retrieve()` method of the `Deployment` API.

In [5]:
openai.Deployment().retrieve('gpt-35-dojo')

<Deployment deployment id=gpt-35-dojo at 0x23c5a859270> JSON: {
  "created_at": 1684879507,
  "id": "gpt-35-dojo",
  "model": "gpt-35-turbo",
  "object": "deployment",
  "owner": "organization-owner",
  "scale_settings": {
    "scale_type": "standard"
  },
  "status": "succeeded",
  "updated_at": 1684917329
}

Using the `Model` API we can also display, for each model, its capabilities. For instance, ChatGPT (gpt-35-turbo) can be used for chat completion, completion and inference.

In [6]:
capabilities = openai.Model.retrieve('gpt-35-turbo').capabilities
print(capabilities)

{
  "chat_completion": true,
  "completion": true,
  "embeddings": false,
  "fine_tune": false,
  "inference": true,
  "scale_types": [
    "standard"
  ]
}


Therefore, all the code written using OpenAI API can be entirely ported into Azure OpenAI at the cost of a lookup using the `openai.Deployment` API to extract the model's name from the deployment object.

## Tokenization
Tokens can be thought of as pieces of words. Before the API processes the prompts, the input is broken down into tokens. These tokens are not cut up exactly where the words start or end - tokens can include trailing spaces and even sub-words. Here are some helpful rules of thumb for understanding tokens in terms of lengths:
- 1 token ~= 4 chars in English
- 1 token ~= ¾ words
- 100 tokens ~= 75 words

How words are split into tokens is language-dependent.
Depending on the model used, requests can use up to 4097 tokens shared between prompt and completion. If your prompt is 4000 tokens, your completion can be 97 tokens at most.
The limit is currently a technical limitation, but there are often creative ways to solve problems within the limit, e.g. condensing your prompt, breaking the text into smaller pieces, etc.

In [7]:
import tiktoken
prompt = """
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline 1: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet's most beloved cooking guru has a buzzy new book and a fresh new perspective
Category: Entertainment

Headline 2: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
n_tokens = len(enc.encode(prompt))
print(n_tokens)

87


## 1. Completion

In [8]:
def get_completion(
        prompt: str,
        deployment_name: str,
        temperature: float = 0.,
        **model_kwargs
) -> str:
    try:
        response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=temperature,
            **model_kwargs
        )
        return response['choices'][0]['text']
    except openai.OpenAIError as e: # this is the base class of any openai exception
        print(f"The call to the Completion API failed as a consequence "
              f"of the following exception: {e}")


compl_deployment_name = 'davinci-3'

In [9]:
prompt = """"
For the below text, provide two labels one each from the following categories:
- Department: “Books”, “Home”, “Fashion”, “Electronics”, “Grocery”, “Others”
- Order intent

Subject: Request for Refund of Recent Book Purchase
Dear [Business Name],
I am writing to request a refund for the books I purchased from your store last week. Unfortunately, the books did not meet my expectations, and I would like to return them for a full refund.
I have attached a copy of the purchase receipt to this email as proof of purchase. The books are in their original packaging and have not been used, so I hope that the refund process will be straightforward.
Please let me know what steps I need to take to return the books and receive a refund. I look forward to hearing back from you soon.
Thank you for your attention to this matter.
Sincerely,
[Your Name]

Response:
"""
response = get_completion(prompt, compl_deployment_name, max_tokens=100)
print(response)

Department: “Books”
Order intent: “Return”


## Chat

In [10]:
def get_chat_completion(
        prompt: str,
        deployment_name: str,
        temperature: float = 0.,
        **model_kwargs
) -> str:
    messages = [{"role": "user", "content": prompt}]
    try:
        response = openai.ChatCompletion.create(
            engine=deployment_name,
            messages=messages,
            temperature=temperature,
            **model_kwargs
        )
        return response['choices'][0]['message']['content']

    except openai.OpenAIError as e: # this is the base class of any openai exception
        print(f"The call to the Chat Completion API failed as a consequence "
              f"of the following exception: {e}")

chat_deployment_name = 'gpt-35-dojo'

In [11]:
text = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""
response = get_chat_completion(prompt, chat_deployment_name, max_tokens=100)
print(response)

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


## Batch prompting

The OpenAI API has separate limits for requests per minute and tokens per minute.

If you're hitting the limit on requests per minute, but have available capacity on tokens per minute, you can increase your throughput by batching multiple tasks into each request. This will allow you to process more tokens per minute, especially with our smaller models.

Sending in a batch of prompts works exactly the same as a normal API call, except you pass in a list of strings to the prompt parameter instead of a single string.

In [12]:
num_stories = 10
prompts = ["Once upon a time,"] * num_stories

# batched example, with 10 story completions per request
response = openai.Completion.create(
    engine=compl_deployment_name,
    prompt=prompts,
    max_tokens=20,
)

# match completions to prompts by index
stories = [""] * len(prompts)
for choice in response.choices:
    stories[choice.index] = prompts[choice.index] + choice.text

# print stories
for story in stories:
    print(story)

Once upon a time, there lived a woman named Maria. She was an excellent cook and would often make delicious meals for her
Once upon a time, there was a prince named Prince Charlie. He lived in a beautiful palace in a faraway land.
Once upon a time, there was a beautiful princess named Aurora. She lived in a castle high in the clouds with her signature
Once upon a time, there was a little girl named Snow White. She had curly blonde hair, eyes as blue as the
Once upon a time, there lived a girl named Alice in a small town in Illinois. She was ten years old, and
Once upon a time, there was a man named Tom who lived in a small cottage in the woods. He had been living
Once upon a time, there was a young girl who lived in a small forest near a mountain. 

One day
Once upon a time, there was a very brave knight. He lived in a small castle with his queen and their two sons
Once upon a time, there existed in a faraway land a magical forest. This forest was populated by many fantastic creatures,

## Chatbot
OpenAI trained the ChatGPT and GPT-4 models to accept input formatted as a conversation. The messages parameter takes an array of dictionaries with a conversation organized by role.

The format of a basic Chat Completion is as follows:

<br/>
<img src="assets/chatbot.png" width="500"/>

The system role also known as the system message is included at the beginning of the array. This message provides the initial instructions to the model.

In [13]:
from typing import List, Dict

def get_chat_with_conversation(
        deployment_name: str,
        messages: List[Dict[str, str]],
        temperature: float = 0.,
        **model_kwargs
) -> str:
    try:
        response = openai.ChatCompletion.create(
            engine=deployment_name,
            messages=messages,
            temperature=temperature,
            **model_kwargs
        )
        return response['choices'][0]['message']['content']
    except openai.OpenAIError as e: # this is the base class of any openai exception
        print(f"The call to the Chat Completion API failed as a consequence "
              f"of the following exception: {e}")

In [18]:
system_role = """
You are a Shakespearean writing assistant who speaks in a Shakespearean style. You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
Here are some example of Shakespeare's style:
 - Romeo, Romeo! Wherefore art thou Romeo?
 - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
 - Shall I compare thee to a summer’s day? Thou art more lovely and more temperate.
"""
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "Hello, my name is Massimiliano from Nice. Can you help me out ?"}
]
response = get_chat_with_conversation(chat_deployment_name, messages)
print(response)

Greetings, Massimiliano from Nice! I shall be delighted to assist thee. What kind of creative content dost thou require assistance with?


One important thing to remember is that, differently from the chat playground we used on Azure cloud, this instance is still **stateless**,  therefore each call is independent and if I call it again asking what is my name it won't remember it:

In [19]:
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "What is my name ?"}
]
response = get_chat_with_conversation(chat_deployment_name, messages)
print(response)

My apologies, kind sir/madam, but I am not privy to your name. Pray, may I know how I can assist thee in your creative endeavors?


However, providing the full context will do the trick:


In [20]:
messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": "Hello, my name is Massimiliano from Nice. Can you help me out ?"},
    {"role": "assistant", "content": "Greetings, Massimiliano from Nice! I shall be delighted to assist thee. What kind of creative content dost thou require assistance with?"},
    {"role": "user", "content": "What is my name ?"}
]
response = get_chat_with_conversation(chat_deployment_name, messages)
print(response)

Thy name is Massimiliano, as thou hast mentioned earlier.
